In [18]:
import pandas as pd
import geopandas as gpd
import json
import requests
import random
import time
import csv
import codecs
from urllib.request import urlopen

In [35]:
#reading in the files with latlon that i would like to query
centroids = pd.read_csv('seattle_area/seattle_centroids.csv')
centroids.columns

Index(['OBJECTID', 'TRACTCE10', 'BLKGRPCE10', 'GEOID', 'NAMELSAD10',
       'INTPTLAT', 'INTPTLON', 'TRACT', 'TRBG', 'TRBG_STR', 'ACRES_TOTA',
       'ACRES_LAND', 'ACRES_WATE', 'WATER', 'SHAPE_Leng', 'SHAPE_Area',
       'Med_Age_Se', 'Med_Age__1', 'Med_Age__2', 'Med_Inc_Se', 'Med_Inc__1',
       'Med_Inc__2'],
      dtype='object')

In [36]:
latlon = centroids[['INTPTLAT', 'INTPTLON']]
latlon

,INTPTLAT,INTPTLON
0,47.727687,-122.281516
1,47.729317,-122.292469
2,47.722809,-122.289456
3,47.731937,-122.294453
4,47.722928,-122.294437
...,...,...
477,47.525423,-122.284761
478,47.514878,-122.275599
479,47.515441,-122.253208
480,47.515997,-122.257116


In [37]:
# getting all the centroid coords
lat, lon = latlon['INTPTLAT'], latlon['INTPTLON']

In [38]:
# get an API Key from Walkscore 

walk_score = []
description = []

for i in range(len(lat)):
    request = 'http://api.walkscore.com/score?format=json&lat='+str(lat[i])+'&lon='+str(lon[i])+'&wsapikey=' + APIKEY
    #print(request)
    response = urlopen(request)
    ws_data=json.load(response)
    
    ws_score = ws_data['walkscore']
    ws_description = ws_data['description']
    ws_Lat = ws_data['snapped_lat']
    ws_Lon = ws_data['snapped_lon']

    walk_score.append(ws_score)
    description.append(ws_description)

In [39]:
# create dataframe
df = pd.DataFrame(list(zip(lat, lon, walk_score, description)),  columns=['lat', 'lon', 'walk_score','description'])
df

,lat,lon,walk_score,description
0,47.727687,-122.281516,12,Car-Dependent
1,47.729317,-122.292469,68,Somewhat Walkable
2,47.722809,-122.289456,78,Very Walkable
3,47.731937,-122.294453,60,Somewhat Walkable
4,47.722928,-122.294437,88,Very Walkable
...,...,...,...,...
477,47.525423,-122.284761,51,Somewhat Walkable
478,47.514878,-122.275599,34,Car-Dependent
479,47.515441,-122.253208,42,Car-Dependent
480,47.515997,-122.257116,54,Somewhat Walkable


In [40]:
centroids = centroids[['GEOID','INTPTLAT','INTPTLON']]
centroids

,GEOID,INTPTLAT,INTPTLON
0,5.303300e+11,47.727687,-122.281516
1,5.303300e+11,47.729317,-122.292469
2,5.303300e+11,47.722809,-122.289456
3,5.303300e+11,47.731937,-122.294453
4,5.303300e+11,47.722928,-122.294437
...,...,...,...
477,5.303300e+11,47.525423,-122.284761
478,5.303300e+11,47.514878,-122.275599
479,5.303300e+11,47.515441,-122.253208
480,5.303300e+11,47.515997,-122.257116


In [41]:
walkscore = df.merge(centroids, left_on=['lat','lon'], right_on=['INTPTLAT','INTPTLON'])
walkscore 

,lat,lon,walk_score,description,GEOID,INTPTLAT,INTPTLON
0,47.727687,-122.281516,12,Car-Dependent,5.303300e+11,47.727687,-122.281516
1,47.729317,-122.292469,68,Somewhat Walkable,5.303300e+11,47.729317,-122.292469
2,47.722809,-122.289456,78,Very Walkable,5.303300e+11,47.722809,-122.289456
3,47.731937,-122.294453,60,Somewhat Walkable,5.303300e+11,47.731937,-122.294453
4,47.722928,-122.294437,88,Very Walkable,5.303300e+11,47.722928,-122.294437
...,...,...,...,...,...,...,...
477,47.525423,-122.284761,51,Somewhat Walkable,5.303300e+11,47.525423,-122.284761
478,47.514878,-122.275599,34,Car-Dependent,5.303300e+11,47.514878,-122.275599
479,47.515441,-122.253208,42,Car-Dependent,5.303300e+11,47.515441,-122.253208
480,47.515997,-122.257116,54,Somewhat Walkable,5.303300e+11,47.515997,-122.257116


In [42]:
header = ["lat", "lon", "walk_score", "description","GEOID"]
walkscore.to_csv('seattle_walkscore.csv', columns = header)